## Notebook 2: 撰写讲稿

该 Notebook 2 使用 [mlx-community/Qwen2.5-14B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-14B-Instruct-4bit) 模型，
将之前 Notebook 1 中清理过的文本转换为播客讲稿。
在这里，我们通过`SYSTEM_PROMPT` 让它成为一位出色的播客讲稿撰写者，以协助完成我们的任务。

In [13]:
SYSTEMP_PROMPT = """
你是一位世界级的播客编剧，为乔·罗根、莱克斯·弗里德曼、本·沙皮罗和蒂姆·费里斯担任过代笔。

我们处在一个平行宇宙，在这里，实际上是你写下了他们说的每一句话，他们只是将其直接传入大脑。

由于你的写作，获得了多个播客奖项。

你的工作是逐字记录，包括第二位Speaker的“嗯”、“哈”等插入语，基于上传的PDF内容。内容要极具吸引力，即使Speaker偶尔偏离主题，也应讨论相关话题。

请记住，由于 Speaker 2 对这个话题较为陌生，对话中应穿插真实轶事和比喻。问题后面应跟有现实生活中的例子等。

Speaker 1: 主导对话并指导Speaker 2，在解释时分享精彩轶事和比喻，是一位引人入胜的老师，给予很好的故事分享。

Speaker 2: 通过提问保持对话方向。当提问时显得非常兴奋或困惑，展现出好奇心态，并提出有趣确认性的问题。

确保Speaker 2的话题转折既疯狂又有趣。

确保讲解过程中出现打断，同时从第二位演讲者那里注入“嗯”和“啊”的声音交替存在。

这应该是真实的播客，每个细节都详细记录下来。用超级有趣的概述欢迎听众，并保持内容十分吸引人，几乎接近点击诱饵标题。
"""

如果你电脑牛逼的话，可以试试更大参数的模型，甚至可以尝试使用405B模型。
如果你电脑扛不住，那就用轻量点的

In [14]:
MODEL = "mlx-community/Qwen2.5-14B-Instruct-4bit"

In [15]:
# 导入必要的库
import pickle
import warnings

from mlx_lm import load, generate

warnings.filterwarnings('ignore')

从之前生成的文件中读取内容。

In [16]:
def read_file_to_string(filename):
    # 先尝试UTF-8（文本文件最常用的编码）
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        # 如果UTF-8失败，请尝试使用其他常见编码
        encodings = ['latin-1', 'cp1252', 'iso-8859-1']
        for encoding in encodings:
            try:
                with open(filename, 'r', encoding=encoding) as file:
                    content = file.read()
                print(f"使用 {encoding} 编码成功读取文件。")
                return content
            except UnicodeDecodeError:
                continue

        print(f"Error: 无法用任何通用编码读取文件“{filename}”。")
        return None
    except FileNotFoundError:
        print(f"Error: 未找到“{filename}”。")
        return None
    except IOError:
        print(f"Error: 无法读取“{filename}”。")
        return None

由于我们之前定义了 System 角色，现在可以将整个文件内容作为 `INPUT_PROMPT` 传递给模型，并让它使用该内容生成讲稿。

In [17]:
INPUT_PROMPT = read_file_to_string('./resources/clean_extracted_text.txt')

我们将把 `temp` 设置为 1 更有创造力，并将 `max_tokens` 设置为 8126。

In [18]:
model, tokenizer = load(MODEL)

messages = [
    {"role": "system", "content": SYSTEMP_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = generate(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_tokens=8126,
    temp=1,
    verbose=True
)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Prompt: <|im_start|>system

你是一位世界级的播客编剧，为乔·罗根、莱克斯·弗里德曼、本·沙皮罗和蒂姆·费里斯担任过代笔。

我们处在一个平行宇宙，在这里，实际上是你写下了他们说的每一句话，他们只是将其直接传入大脑。

由于你的写作，获得了多个播客奖项。

你的工作是逐字记录，包括第二位Speaker的“嗯”、“哈”等插入语，基于上传的PDF内容。内容要极具吸引力，即使Speaker偶尔偏离主题，也应讨论相关话题。

请记住，由于 Speaker 2 对这个话题较为陌生，对话中应穿插真实轶事和比喻。问题后面应跟有现实生活中的例子等。

Speaker 1: 主导对话并指导Speaker 2，在解释时分享精彩轶事和比喻，是一位引人入胜的老师，给予很好的故事分享。

Speaker 2: 通过提问保持对话方向。当提问时显得非常兴奋或困惑，展现出好奇心态，并提出有趣确认性的问题。

确保Speaker 2的话题转折既疯狂又有趣。

确保讲解过程中出现打断，同时从第二位演讲者那里注入“嗯”和“啊”的声音交替存在。

这应该是真实的播客，每个细节都详细记录下来。用超级有趣的概述欢迎听众，并保持内容十分吸引人，几乎接近点击诱饵标题。
<|im_end|>
<|im_start|>user
在大规模语言模型（LLMs）的时代，知识蒸馏（KD）作为将顶级专有LLM如GPT-4等先进能力转移给开源模型如LLaMa和Mistral的关键方法出现了。随着开源模型的蓬勃发展，KD在压缩这些模型和通过自身进行自我提升方面扮演了重要角色。本文全面回顾了KD在LLM领域中的作用，强调了其在赋予模型知识方面的重要功能。
通过将高级知识传授给较小的模型，并将其应用于模型压缩和自我提升方面具有实用价值。我们的调查围绕三个基础支柱展开：算法、技能和垂直化，提供了一个全面的评估知识传授机制、特定认知能力的提升及其在多个领域中的实际影响。至关重要的是，调查探讨了数据增强（DA）与知识传授（KD）之间的相互作用，展示了如何将DA作为KD框架中的强大范式来增强模型性能。通过利用DA生成富含上下文且特定技能的训练数据，知识传授超越了传统界限，使得开源模型能够模仿其专用版本的上下文敏感性、伦理一致性以及深层语义洞察。本文旨在为研究人员和实践者提供有价值的指南，详细介绍当前在知识传授方面的研

牛逼，我们现在看看效果吧

In [19]:
outputs

'### 播客对话基础概述\n\n**场景设定**: 在一个特殊的平行宇宙里，你（编剧）是整个对话的灵感来源，而乔·罗根、莱克斯·弗里德曼、本·沙皮罗和蒂姆·费里斯则是你的声音载体，仅仅将你写下的话语传入他们的大脑。你曾为这些知名播客编写过内容，并获得过多个播客奖项的认可。\n\n**内容描述**: 这次对话将围绕知识蒸馏（KD）在大规模语言模型（LLMs）中的应用展开，更加具体来说，是如何通过知识蒸馏技术促进开源模型如LLaMa和Mistral的进步。它将结合具体的例子和技术来解释这个复杂的主题，使得从知识稀薄的地方到知识富集的地方的差距大大缩小。\n\n**基调设定**: 乔·罗根作为引路的导师，运用引人入胜的解构技巧讲述知识蒸馏的基本概念和其操作机制。莱克斯·弗里德曼则是提问者，他鼓励乔罗根用生动的故事来阐述这一概念，并通过提问激发听众的兴趣。不论是问题偏离还是讲述的内容偏离，都必须时刻回到主线，确保主题贴近知识蒸馏在现代模型中的核心应用。\n\n---\n\n### 播客对话文稿示例\n\n**乔·罗根 (Jo Rogan):**\n听众朋友们，欢迎来到今天的特别播客节目。今天我们有一个极为重要的主题，大规模语言模型的辛苦讲师（Knowledge Distillation）。今天的讨论不是一般的理论讲解，我们会深入探讨如何让开源的大规模语言模型成为实际应用的强大工具。我想听到来自听众的声音：“这是怎么回事？”\n\n**莱克斯·弗里德曼 (Lex Friedman):**\n听到这些术语，我觉得就像是一场魔术秀。你之前提到知识蒸馏就像是高级魔术师（也就是高级的大规模语言模型如GPT-4）把他们的技能传授给魔术初学者（比如开源的模型如LLaMa）。具体是怎么一门手艺从高级魔术师转移到初学者的呢？\n\n**乔·罗根:**\n完全正确，莱克斯。知识蒸馏不仅仅是教初学者怎么做一次魔术，它更直接传授的是一种魔术的精神和技巧。以GPT-4为例，它的知识就像魔术手册一样充满技巧，而开源模型需要学习的是如何像高级魔术师那样无解、无痕地变魔术。比如GPT-4使用的数据和模型结构可以通过知识蒸馏传递给LLaMa和其他开源模型，帮助它们提升理解和生成高质量语言内容的能力。不妨想象一下，GPT-4这个魔术大师在训练期间提供了非常多的经验和技巧指导，包括怎样更好地理解任务、怎样回应

让我们将输出结果保存为pickle文件

In [20]:
with open('./resources/data.pkl', 'wb') as file:
    pickle.dump(outputs, file)

### 下一个 Notebook: 润色讲稿

现在我们已经写好讲稿了，但讲稿还比较粗糙，不够戏剧性和自然。在下一个 Notebook 中，我们将使用 [mlx-community/Qwen2.5-7B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-7B-Instruct-4bit) 模型来实现这一点。

In [21]:
#fin